In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv('data.csv')
df.head()

In [ ]:
df['class'].value_counts()

In [ ]:
import random

class_counts = df['class'].value_counts()

min_count = min(class_counts)

indices_to_remove = random.sample(df[df['class'] == 0].index.tolist(), class_counts[0] - min_count)

df = df.drop(indices_to_remove)

df['class'].value_counts()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.isnull().value_counts()

In [ ]:
df.hist(figsize=(15,15))

In [ ]:
sns.heatmap(df.corr(),annot=True)

In [ ]:
df.corrwith(df['class']).abs().sort_values(ascending=False)

In [ ]:
X = df[df.columns.difference(['class'])]
y = df[['class']]

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(X,y ,random_state=0,test_size=0.2)

In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

In [ ]:
print(clf.coef_)
print(clf.intercept_)

In [ ]:
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("Precision: ",metrics.precision_score(y_test, y_pred))
print("Recall: " ,metrics.recall_score(y_test, y_pred))
print("F1 score: " ,metrics.f1_score(y_test, y_pred))

In [ ]:
metrics.confusion_matrix(y_test,y_pred)

In [ ]:
variance = 3.116
skewness = 8.5661
curtosis = -2.573
entropy = -0.54699

clf.predict([[variance,skewness,curtosis,entropy]])[0]

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_test)

clfPCA = svm.SVC(kernel='linear')
clfPCA.fit(X_pca, y_test)

x_min, x_max = X_pca[:, 0].min() - 1, X_pca[:, 0].max() + 1
y_min, y_max = X_pca[:, 1].min() - 1, X_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max,0.01), np.arange(y_min, y_max,0.01))

Z = clfPCA.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X_pca[:, 0], X_pca[:, 1], cmap='coolwarm')
plt.show()